In [18]:
#from pose_estimator_2d import openpose_estimator
from utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import cv2
import importlib
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML

## Estimate 2D pose from video

In [19]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time

# Function to draw landmarks on the image
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style())

        # Add numbers beside each landmark
        for i, landmark in enumerate(pose_landmarks):
            x = int(landmark.x * annotated_image.shape[1])
            y = int(landmark.y * annotated_image.shape[0])
            cv2.putText(annotated_image, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    return annotated_image

In [70]:
# Initialize the new MediaPipe Pose detector
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# Open the video file
cap = cv2.VideoCapture('miscs/cxk.mp4')

# Define the MediaPipe to OpenPose joint mapping (33 to 25)
mediapipe_to_openpose = {
    0: 0,  # Nose
    11: 11,  # Left Hip
    12: 12,  # Right Hip
    23: 13,  # Left Knee
    24: 14,  # Right Knee
    25: 15,  # Left Ankle
    26: 16,  # Right Ankle
    27: 17,  # Left Heel
    28: 18,  # Right Heel
    31: 19,  # Left Foot Index
    32: 20,  # Right Foot Index
    13: 5,  # Left Shoulder
    14: 6,  # Right Shoulder
    15: 7,  # Left Elbow
    16: 8,  # Right Elbow
    17: 9,  # Left Wrist
    18: 10,  # Right Wrist
}

# Process video frames and extract pose landmarks
keypoints_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_height, img_width = frame.shape[:2]
    # Convert the image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect pose landmarks
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    detection_result = detector.detect(mp_image)

    if detection_result.pose_landmarks:
        keypoints = np.zeros((25, 3))  # OpenPose has 25 keypoints

        # Map MediaPipe landmarks to OpenPose 25 keypoints
        for mp_idx, openpose_idx in mediapipe_to_openpose.items():
            if mp_idx < len(detection_result.pose_landmarks[0]):
                landmark = detection_result.pose_landmarks[0][mp_idx]
                keypoints[openpose_idx] = [landmark.x, landmark.y, landmark.z]

        keypoints_list.append(keypoints)
    else:
        keypoints_list.append(None)

    annotated_image = draw_landmarks_on_image(rgb_frame, detection_result)

    # Resize the visualization window
    cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)  # Make the window resizable
    cv2.resizeWindow('Pose Detection', 1500, 750)  # Set the window size 

    cv2.imshow('Pose Detection', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

    if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()

# Save keypoints_list or process further
print("Pose detection completed.")


c:\Python\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Pose detection completed.


In [ ]:
print(keypoints_list)

## Process 2D pose

In [51]:
# save 2d pose result
pose2d = np.stack(keypoints_list)[:, :, :2]
pose2d_file = Path('2d_pose.npy')
np.save(pose2d_file, pose2d)

In [21]:
# Load the data from the .npy file
data = np.load('miscs/cxk_cache/2d_pose.npy')

# Print the shape of the data
print("Shape of the data:", data.shape)

Shape of the data: (900, 25, 2)


In [52]:
# Load the data from the .npy file
data = np.load('2d_pose.npy')

# Print the shape of the data
print("Shape of the data:", data.shape)

Shape of the data: (900, 25, 2)


## Visualize 2D pose

In [17]:
from pathlib import Path

# Set the path to save the visualized images
vis_result_dir = Path('2d_pose_vis')

# Ensure the directory exists
vis_result_dir.mkdir(parents=True, exist_ok=True)

cap = cv2.VideoCapture(str('trial.mp4'))
op_skel = openpose_skeleton.OpenPoseSkeleton()

for i, keypoints in enumerate(keypoints_list):
    ret, frame = cap.read()
    if not ret:
        break
    
    # keypoint whose detect confidence under kp_thresh will not be visualized
    output_file = vis_result_dir / f'{i:04d}.png'  # Correct path joining
    vis.vis_2d_keypoints(
        keypoints=keypoints,
        img=frame,
        skeleton=op_skel,
        kp_thresh=0.4,
        output_file=str(output_file)  # Convert to string if necessary
    )

cap.release()


## Initialize 3D pose estimator (Still working on it)

In [53]:
import torch
import pathlib
import importlib
from pose_estimator_3d import estimator_3d 

# Fix pathlib.PosixPath issue on Windows
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Reload the `estimator_3d` module to ensure it's fresh
importlib.reload(estimator_3d)

# Initialize Estimator3D with the configuration and checkpoint
e3d = estimator_3d.Estimator3D(
    config_file='models/video_pose.yaml',
    checkpoint_file='models/best_58.58.pth'
)

# Revert pathlib changes (optional)
pathlib.PosixPath = temp

# (Optional) Test the e3d object to ensure it works
print("e3d initialized:", e3d)


=> Read 3D estimator config from C:\Users\dodom\OneDrive - Misr International University\Desktop\College\college\Graduation Project\video2bvh-master\video2bvh-master\models\video_pose.yaml.
{'DATASET': {'CAM_PARAMS': '/home/kevin/public98/3dpose/Dataset/h36m/cameras.h5',
             'IMAGE_HEIGHT': 1002,
             'IMAGE_WIDTH': 1000,
             'INPUT_LEFT_JOINTS': [5, 6, 7, 12, 13, 14, 16, 18, 19, 20, 21],
             'INPUT_RIGHT_JOINTS': [2, 3, 4, 9, 10, 11, 15, 17, 22, 23, 24],
             'INPUT_ROOT': '/home/kevin/HDD/h36m_dataset/2D_openpose',
             'IN_CHANNEL': 2,
             'IN_JOINT': 25,
             'NAME': 'h36m',
             'OUTPUT_LEFT_JOINTS': [4, 5, 6, 11, 12, 13],
             'OUTPUT_RIGHT_JOINTS': [1, 2, 3, 14, 15, 16],
             'OUT_CHANNEL': 3,
             'OUT_JOINT': 17,
             'SEQ_LEN': 243,
             'TARGET_ROOT': '/home/kevin/HDD/h36m_dataset/3D_gt',
             'TEST_FLIP': True,
             'TRAIN_FLIP': True},
 'MODEL

## Estimate 3D pose from 2D pose (still working on it)

In [55]:
pose2d = np.load('2d_pose.npy')
pose3d = e3d.estimate(pose2d, image_width=img_width, image_height=img_height)

=> Begin to estimate 3D poses.
900 / 900


In [25]:
# Load the data from the .npy file
data3d = np.load('miscs/cxk_cache/3d_pose.npy')

# Print the shape of the data
print("Shape of the data:", data3d.shape)

Shape of the data: (900, 17, 3)


In [67]:
# Load the data from the .npy file
pose3d = np.load('3d_pose.npy')

# Print the shape of the data
print("Shape of the data:", pose3d.shape)

Shape of the data: (900, 17, 3)


In [68]:
pose3d_world = pose3d  # Data is already in world coordinates

pose3d_file =  '3d_pose.npy'
np.save(pose3d_file, pose3d_world)

## Convert 3D pose to BVH

In [69]:
bvh_file = f'{"hellococo"}.bvh'
cmu_skel = cmu_skeleton.CMUSkeleton()
channels, header = cmu_skel.poses2bvh(pose3d_world, output_file=bvh_file)
print("Converted succesfully")

Converted succesfully


In [6]:
import os
import subprocess
import logging

logging.basicConfig(level=logging.DEBUG)

def open_blender_with_bvh(
    bvh_file,
    blender_executable=r"C:/Program Files/Blender Foundation/Blender 3.5/blender.exe"
):
    """
    Opens Blender, imports a BVH file, and ensures armature visibility.
    
    Parameters:
        bvh_file (str): Path to the BVH file to be imported.
        blender_executable (str): Path to the Blender executable.
    """
    # Check if the BVH file exists
    if not os.path.exists(bvh_file):
        raise FileNotFoundError(f"The BVH file '{bvh_file}' does not exist.")
    
    # Create a temporary Python script to run in Blender
    blender_script = f"""
import bpy
import os

# Path to the BVH file
bvh_file = r"{os.path.abspath(bvh_file)}"

def import_bvh(filepath, scale=1.0):
    # Ensure everything is deselected
    bpy.ops.object.select_all(action='DESELECT')

    # Import the BVH file
    bpy.ops.import_anim.bvh(
        filepath=filepath,
        axis_forward='-Z',   # Adjust if necessary
        axis_up='Y',         # Adjust if necessary
        filter_glob="*.bvh",
        target='ARMATURE',
        global_scale=scale,
        frame_start=1,
        use_fps_scale=True
    )

def set_armature_display():
    for obj in bpy.context.scene.objects:
        if obj.type == 'ARMATURE':
            bpy.context.view_layer.objects.active = obj  # Make it the active object
            obj.select_set(True)
            # Ensure it's visible in the viewport
            obj.hide_viewport = False
            obj.hide_set(False)
            # Change display mode for better visibility
            obj.data.display_type = 'STICK'  # Other options: 'ENVELOPE', 'OCTAHEDRAL', etc.

# Clear existing scene
bpy.ops.wm.read_factory_settings(use_empty=True)

# Import BVH file
import_bvh(bvh_file, scale=10.0)

# Adjust armature visibility and display settings
set_armature_display()

# Optionally save the Blender file
output_blend = os.path.splitext(bvh_file)[0] + ".blend"
bpy.ops.wm.save_as_mainfile(filepath=output_blend)

print("BVH file imported and processed successfully.")
    """
    
    # Write the Blender script to a temporary file
    script_file = "import_bvh_to_blender.py"
    with open(script_file, "w") as file:
        file.write(blender_script)
    
    # Run Blender with the temporary script
    try:
        subprocess.run([blender_executable, "--python", script_file], check=True)
    finally:
        # Clean up the temporary script
        if os.path.exists(script_file):
            os.remove(script_file)

# Example usage
bvh_file = "miscs/cxk_cache/cxk.bvh"  # Replace with the correct path to your BVH file
try:
    open_blender_with_bvh(bvh_file)
except Exception as e:
    logging.error(f"Failed to process BVH file: {e}")


In [7]:
output = 'miscs/h36m_cxk.bvh'
h36m_skel = h36m_skeleton.H36mSkeleton()
_ = h36m_skel.poses2bvh(pose3d_world, output_file=output)